# Benchmark with other quantum chemistry output file parser projects

As far as I know there are several mature parsers offerring python API for quantum chemistry output files, which have been in development for several years. 

1. [pymatgen](https://pymatgen.org/pymatgen.io.qchem.html)
2. [cclib](https://cclib.github.io/)
3. [openeye](https://docs.eyesopen.com/toolkits/python/oechemtk/molreadwrite.html) 

*Attention*: The project [openeye](https://docs.eyesopen.com/toolkits/python/oechemtk/molreadwrite.html)  is supposed to be powerful from its documentation, but unfortunately I tried to run the sample code and it crashed the kernel, so I can't compare it here.

The main functions of the above projects are not exactly the same as those of MolOP. This benchmark only covers performance and functionality comparisons of our completed parsers. Our project currently only supports parsing of Gaussian and xTB related files, while the other projects have extensive support for various quantum chemistry software. We may add support for other software formats in the future, but for now, if you need parsing of other software outputs, you can use the mature projects above to accomplish your goals.

In fact, I didn't realise these projects existed until I decided to develop MolOP. If I had known about them earlier, perhaps the MolOP project would have been another tool that simply provided a way to go **from xyz coordinates to molecular graphs**. Of course, now you can also use just this functionality, I have wrapped a separate function for it and you can learn about its contribution at [**structure_recover_benchmark**](structure_recover_benchmark.md).

In [1]:
import time
from glob import glob
from molop import AutoParser
from molop.config import molopconfig
from pymatgen.io import gaussian
from cclib.io import ccopen


def timer_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        run_time = end_time - start_time
        print(f"Function '{func.__name__}' ran in {run_time:.6f} seconds")
        return result

    return wrapper


@timer_decorator
def pymatgen_parser(file_wildcard: str):
    result = []
    for file in glob(file_wildcard):
        try:
            result.append(gaussian.GaussianOutput(file))
        except Exception as e:
            print(f"Error parsing file {file}: {e}")
    return result


@timer_decorator
def cclib_parser(file_wildcard: str):
    result = []
    for file in glob(file_wildcard):
        try:
            result.append(ccopen(file).parse())
        except Exception as e:
            print(f"Error parsing file {file}: {e}")
    return result


@timer_decorator
def molop_parser(file_wildcard: str, quiet: bool = False, **kwargs):
    if quiet:
        molopconfig.quiet()
    else:
        molopconfig.verbose()
    return AutoParser(file_wildcard, **kwargs)

## Benchmark files

The benchmark files are located in the `tests/test_files` directory. All of them are legal Gaussian output files.

In [2]:
print(AutoParser("../../tests/test_files/g16log/*.log").to_summary_df().to_markdown())

MolOP parsing with 16 jobs: 100%|██████████| 66/66 [00:02<00:00, 32.46it/s]
0 files failed to parse, 66 successfully parsed


|    | parser            | file_path                                                                                                                 | file_name                                                                    | file_format   | version                                    |   frame_index |   charge |   multiplicity | SMILES                                                                                                                                                                                                                            | functional   | basis           | solvent_model   | solvent      |   temperature | status                                                                                                                                                                                                   | is_optimized   |           sp |      ZPE |      TCE |      TCH |       TCG |          U_0 |          U_T |          H_T |          G_T |      Cv |   ent

## Time benchmark

In short, MolOP is up to 20 times faster than pymatgen, and up to 40 times faster than cclib.

In [3]:
pymatgen_result = pymatgen_parser("../../tests/test_files/g16log/*.log")

Error parsing file ../../tests/test_files/g16log/anion_0167_opt_g16_nbo_sp.log: list index out of range


/home/tmj/miniconda3/envs/molop/lib/python3.8/site-packages/pymatgen/io/gaussian.py:1129: UserWarning: 
../../tests/test_files/g16log/cation_0407_opt_g16.log: Termination error or bad Gaussian output file !
  warnings.warn("\n" + self.filename + ": Termination error or bad Gaussian output file !")


Error parsing file ../../tests/test_files/g16log/radical_0554_opt_g16_nbo_sp.log: list index out of range
Function 'pymatgen_parser' ran in 3.101514 seconds


pymatgen took about 3.1s and raised 2 errors

In [4]:
cclib_result = cclib_parser("../../tests/test_files/g16log/*.log")

[Gaussian ../../tests/test_files/g16log/dsgdb9nsd_000484-1+.log ERROR] Encountered error when parsing.
[Gaussian ../../tests/test_files/g16log/dsgdb9nsd_000484-1+.log ERROR] Last line read:  Rotational constants (GHZ):      ************     1.38922     1.38922



Error parsing file ../../tests/test_files/g16log/dsgdb9nsd_000484-1+.log: could not convert string to float: '************'
Function 'cclib_parser' ran in 5.726074 seconds


cclib took about 5.7s and raised 1 errors

In [5]:
molop_result = molop_parser("../../tests/test_files/g16log/*.log", n_jobs=1)

MolOP parsing with single thread: 100%|██████████| 66/66 [00:03<00:00, 18.05it/s]
0 files failed to parse, 66 successfully parsed


Function 'molop_parser' ran in 3.659240 seconds


Sequential MolOP took about 3.6s and raised no error. We can be faster by the automatic parallelization.

In [6]:
molop_result = molop_parser("../../tests/test_files/g16log/*.log")

MolOP parsing with 16 jobs: 100%|██████████| 66/66 [00:00<00:00, 92.74it/s]
0 files failed to parse, 66 successfully parsed


Function 'molop_parser' ran in 0.716954 seconds


Parallel MolOP took about 0.7s and raised no error. We can be faster if we want only the optimized structure.

In [7]:
molop_result = molop_parser(
    "../../tests/test_files/g16log/*.log", only_last_frame=True)

MolOP parsing with 16 jobs: 100%|██████████| 66/66 [00:00<00:00, 502.50it/s]
0 files failed to parse, 66 successfully parsed


Function 'molop_parser' ran in 0.138054 seconds


Parallel MolOP to extract only the last frame took close to 0.1s and raised no error.

## Information benchmark

In [12]:
cclib_result[-1].getattributes().keys()

dict_keys(['atomcharges', 'atomcoords', 'atommasses', 'atomnos', 'charge', 'coreelectrons', 'dispersionenergies', 'enthalpy', 'entropy', 'freeenergy', 'geotargets', 'geovalues', 'grads', 'homos', 'metadata', 'moenergies', 'moments', 'mosyms', 'mult', 'natom', 'nbasis', 'nmo', 'optdone', 'optstatus', 'polarizabilities', 'pressure', 'rotconsts', 'scfenergies', 'scftargets', 'scfvalues', 'temperature', 'vibdisps', 'vibfconsts', 'vibfreqs', 'vibirs', 'vibrmasses', 'vibsyms', 'zpve'])

In [13]:
pymatgen_result[-1].as_dict().keys()

dict_keys(['has_gaussian_completed', 'nsites', 'unit_cell_formula', 'reduced_cell_formula', 'pretty_formula', 'is_pcm', 'errors', 'Mulliken_charges', 'elements', 'nelements', 'charge', 'spin_multiplicity', 'input', 'output', '@module', '@class'])

In [14]:
molop_result[-1][-1].to_dict().keys()

dict_keys(['filename', 'frame_id', 'smiles', 'atom_number', 'total_charge', 'total_multiplicity', 'atoms', 'coords', 'standard_coords', 'bonds', 'formal_charges', 'formal_num_radicals', 'qm_software', 'qm_software_version', 'functional', 'basis', 'solvent_model', 'solvent', 'temperature', 'keywords', 'mulliken_charge', 'spin_densities', 'gradients', 'single_point_energy', 'zpve', 'energy_correction', 'enthalpy_correction', 'gibbs_free_energy_correction', 'U_0', 'U_T', 'H_T', 'G_T', 'S', 'Cv', 'alpha_orbital_energies', 'beta_orbital_energies', 'alpha_homo', 'alpha_lumo', 'alpha_gap', 'beta_homo', 'beta_lumo', 'beta_gap', 'freqs', 'IR_intensities', 'force_constants', 'vibration_mode', 'frequence_analysis', 'is_TS', 'is_optimized', 'is_error', 'spin_quantum_number', 'spin_square', 'rotation_consts', 'isotropic_polarizability', 'electronic_spatial_extent', 'dipole', 'quadrupole', 'octapole', 'hexadecapole', 'nbo_bond_order', 'wiberg_bond_order', 'mo_bond_order', 'atom_atom_overlap_bond_ord

pymatgen extracts only the basic information from the Gaussian output file.

It's great that cclib has a very full parsing of the Gaussian output file, covering almost everything in it. Unfortunately there are bugs in the handling of some fields.

MolOP's support for Gaussian output files exceeds that of pymatgen, but hardly exceeds the coverage of cclib. 

More importantly, the smallest unit of data extracted by MolOP is each legal frame in a file, and each frame is parsed to the same specification. Other projects will only process a file as a whole. The structure of the optimisation process is equally important for tasks such as building datasets of molecular geometries. Each frame in the geometry optimisation is a legitimate SCF convergent structure and provides a DFT-level molecular force field, which may be of great help for neural network potential function fitting.

In [16]:
molop_result = molop_parser("../../tests/test_files/g16log/*.log")
print(molop_result[10].to_summary_df().to_markdown(floatfmt=".6f"))

MolOP parsing with 16 jobs: 100%|██████████| 66/66 [00:00<00:00, 90.57it/s]
0 files failed to parse, 66 successfully parsed


Function 'molop_parser' ran in 0.732880 seconds
|    | parser            | file_path                                                                              | file_name                                 | file_format   | version                                    |   frame_index |   charge |   multiplicity | SMILES                                            | functional   | basis   | solvent_model   | solvent   |   temperature | status                                                                                                                                                                                                   | is_optimized   |           sp |      ZPE |      TCE |      TCH |      TCG |          U_0 |          U_T |          H_T |          G_T |        Cv |    entropy |      HOMO |      LUMO |      GAP |   first freq | first freq tag   |   second freq | second freq tag   | S   | S**2   | rotational_constants            |   electronic_spatial_extent |   isotropic_pola